In [89]:
import submission
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
import math

In [109]:
## Parameters settings
past_cases_interval = 10
past_weather_interval = 10


## Read training data
train_file = './data/COVID_train_data.csv'
train_df = pd.read_csv(train_file)

## Read Training labels
train_label_file = './data/COVID_train_labels.csv'
train_labels_df = pd.read_csv(train_label_file)


## Read testing Features
test_fea_file = './data/test_features.csv'
test_features = pd.read_csv(test_fea_file)

## Set hyper-parameters for the SVM Model
svm_model = SVR()
svm_model.set_params(**{'kernel': 'rbf', 'degree': 1, 'C': 5000,
                        'gamma': 'scale', 'coef0': 0.0, 'tol': 0.001, 'epsilon': 10})

train_df.shape

#ideal is that:
# we use MAX interval days 30, to constract a feature matrix
# select the target feature matrix by using the defined past_cases and past_weather interval


(192, 18)

In [73]:
feature_matrix_y_for_max_interval = train_labels_df[30:]

In [76]:
# only start from day 30 for max interval 
# so it can get the past days
draft_df = train_df.copy()
to_expand_list = draft_df.columns[1:]
expanding_list = []

for word in to_expand_list:
    for i in range(1,31):
        to_add = str(word+"-{}".format(i))
        expanding_list.append(to_add)

# feature_matrix_X_for_max_interval = pd.DataFrame(columns=expanding_list)

i = 30
add_rows = []
while i < 192:
    index = i-30
    row = []
    for word in expanding_list:
        feature,day_number = word.split('-')
#         row.append(train_df.iloc[index+int(day_number)-1][feature]) # this is 1 is 1st day
        row.append(train_df.iloc[i-int(day_number)][feature]) # this is 1 is 29st day
    add_rows.append(row)
    i += 1


In [77]:
feature_matrix_X_for_max_interval = pd.DataFrame(data=add_rows, columns=expanding_list)

In [94]:
feature_matrix_X_for_max_interval

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Columns: 510 entries, max_temp-1 to dailly_cases-30
dtypes: float64(510)
memory usage: 645.6 KB


In [79]:
feature_matrix_y_for_max_interval

,day,dailly_cases
30,31,0
31,32,0
32,33,0
33,34,0
34,35,0
...,...,...
187,188,1020
188,189,861
189,190,923
190,191,1151


In [80]:
selected_keywords = ['max_temp', 'max_dew', 'max_humid', 'dailly_cases']
selected_columns = []
for word in selected_keywords:
    i = 1
    if (word == 'dailly_cases'):
        while (i <= past_cases_interval):
            selected_columns.append(word+"-{}".format(i))
            i +=1
        
    else:
        while (i <= past_weather_interval):
            selected_columns.append(word+"-{}".format(i))
            i +=1

In [81]:
part_1_feature_X = feature_matrix_X_for_max_interval[selected_columns]

In [82]:
part_1_feature_X

,max_temp-1,max_temp-2,max_temp-3,max_temp-4,max_temp-5,max_temp-6,max_temp-7,max_temp-8,max_temp-9,max_temp-10,...,dailly_cases-1,dailly_cases-2,dailly_cases-3,dailly_cases-4,dailly_cases-5,dailly_cases-6,dailly_cases-7,dailly_cases-8,dailly_cases-9,dailly_cases-10
0,36.0,47.0,45.0,51.0,46.0,31.0,40.0,43.0,45.0,47.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,36.0,36.0,47.0,45.0,51.0,46.0,31.0,40.0,43.0,45.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,50.0,36.0,36.0,47.0,45.0,51.0,46.0,31.0,40.0,43.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,57.0,50.0,36.0,36.0,47.0,45.0,51.0,46.0,31.0,40.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,64.0,57.0,50.0,36.0,36.0,47.0,45.0,51.0,46.0,31.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,95.0,91.0,81.0,91.0,94.0,93.0,95.0,96.0,94.0,86.0,...,771.0,944.0,1032.0,1155.0,846.0,991.0,967.0,738.0,964.0,1072.0
158,97.0,95.0,91.0,81.0,91.0,94.0,93.0,95.0,96.0,94.0,...,1020.0,771.0,944.0,1032.0,1155.0,846.0,991.0,967.0,738.0,964.0
159,96.0,97.0,95.0,91.0,81.0,91.0,94.0,93.0,95.0,96.0,...,861.0,1020.0,771.0,944.0,1032.0,1155.0,846.0,991.0,967.0,738.0
160,90.0,96.0,97.0,95.0,91.0,81.0,91.0,94.0,93.0,95.0,...,923.0,861.0,1020.0,771.0,944.0,1032.0,1155.0,846.0,991.0,967.0


In [83]:
part_1_test_features_X = test_features[selected_columns]

In [84]:
part_1_test_features_X

,max_temp-1,max_temp-2,max_temp-3,max_temp-4,max_temp-5,max_temp-6,max_temp-7,max_temp-8,max_temp-9,max_temp-10,...,dailly_cases-1,dailly_cases-2,dailly_cases-3,dailly_cases-4,dailly_cases-5,dailly_cases-6,dailly_cases-7,dailly_cases-8,dailly_cases-9,dailly_cases-10
0,84,95,90,96,97,95,91,81,91,94,...,1043,1151,923,861,1020,771,944,1032,1155,846
1,89,84,95,90,96,97,95,91,81,91,...,907,1043,1151,923,861,1020,771,944,1032,1155
2,90,89,84,95,90,96,97,95,91,81,...,674,907,1043,1151,923,861,1020,771,944,1032
3,92,90,89,84,95,90,96,97,95,91,...,971,674,907,1043,1151,923,861,1020,771,944
4,82,92,90,89,84,95,90,96,97,95,...,957,971,674,907,1043,1151,923,861,1020,771
5,91,82,92,90,89,84,95,90,96,97,...,831,957,971,674,907,1043,1151,923,861,1020
6,82,91,82,92,90,89,84,95,90,96,...,1124,831,957,971,674,907,1043,1151,923,861
7,79,82,91,82,92,90,89,84,95,90,...,903,1124,831,957,971,674,907,1043,1151,923
8,85,79,82,91,82,92,90,89,84,95,...,880,903,1124,831,957,971,674,907,1043,1151
9,91,85,79,82,91,82,92,90,89,84,...,676,880,903,1124,831,957,971,674,907,1043


In [85]:
svm_model.fit(part_1_feature_X, feature_matrix_y_for_max_interval["dailly_cases"])

SVR(C=5000, degree=1, epsilon=10)

In [86]:
a = svm_model.predict(part_1_test_features_X)

In [92]:
result = []
for val in a:
    result.append(math.floor(val))

print(result)

[945, 897, 832, 881, 907, 921, 1028, 819, 812, 809, 860, 845, 837, 898, 861, 811, 846, 839, 855, 892]


In [ ]:
def get_training_data_for_max_interval(train_df, train_labels_df):
    # get training labels 
    # size 162x2
    feature_matrix_y_for_max_interval = train_labels_df[30:]
    
    # only start from day 30 for max interval 
    # so it can get the past days
    draft_df = train_df.copy()

    # except the day 
    # all the remaining needs to have past values
    to_expand_list = draft_df.columns[1:]
    expanding_list = []
    for word in to_expand_list:
        for i in range(1,31):
            to_add = str(word+"-{}".format(i)) # into format like max_temp-1
            expanding_list.append(to_add)

    # create the data with 30days past records
    i = 30
    add_rows = []
    while i < 192:
        index = i-30
        row = []
        for word in expanding_list:
            feature,day_number = word.split('-')
            row.append(train_df.iloc[i-int(day_number)][feature]) # this is max_temp-1 is 1 day before the target day
        add_rows.append(row)
        i += 1

    # create the dataframe which has the max_interval records
    # size: 162x510
    feature_matrix_X_for_max_interval = pd.DataFrame(data=add_rows, columns=expanding_list)
    
    return feature_matrix_X_for_max_interval, feature_matrix_y_for_max_interval

In [106]:
def predict_COVID_part1(svm_model, train_df, train_labels_df, past_cases_interval, past_weather_interval, test_feature):
    
    # get training labels 
    # size 162x2
    feature_matrix_y_for_max_interval = train_labels_df[30:]
    
    # only start from day 30 for max interval 
    # so it can get the past days
    draft_df = train_df.copy()

    # except the day 
    # all the remaining needs to have past values
    to_expand_list = draft_df.columns[1:]
    expanding_list = []
    for word in to_expand_list:
        for i in range(1,31):
            to_add = str(word+"-{}".format(i)) # into format like max_temp-1
            expanding_list.append(to_add)

    # create the data with 30days past records
    i = 30
    add_rows = []
    while i < 192:
        index = i-30
        row = []
        for word in expanding_list:
            feature,day_number = word.split('-')
            row.append(train_df.iloc[i-int(day_number)][feature]) # this is max_temp-1 is 1 day before the target day
        add_rows.append(row)
        i += 1

    # create the dataframe which has the max_interval records
    # size: 162x510
    feature_matrix_X_for_max_interval = pd.DataFrame(data=add_rows, columns=expanding_list)

    # For part-1, we only need the ['max_temp', 'max_dew', 'max_humid', 'dailly_cases']
    # And with a preset past_cases_interval and past_weather_interval
    selected_keywords = ['max_temp', 'max_dew', 'max_humid', 'dailly_cases']
    selected_columns = []
    for word in selected_keywords:
        i = 1
        if (word == 'dailly_cases'):
            while (i <= past_cases_interval):
                selected_columns.append(word+"-{}".format(i))
                i +=1
            
        else:
            while (i <= past_weather_interval):
                selected_columns.append(word+"-{}".format(i))
                i +=1

    # get the part1 training data
    part_1_feature_X = feature_matrix_X_for_max_interval[selected_columns]

    # get the part1 testing data
    part_1_test_features_X = test_feature[selected_columns].values.reshape(1, -1)
    
    # fit data to the SVM model
    # do predicting
    svm_model.fit(part_1_feature_X, feature_matrix_y_for_max_interval["dailly_cases"])
    a = svm_model.predict(part_1_test_features_X)

    # we only take the floor integer of the predicted value
    # result = []
    # for val in a:
    #     result.append(math.floor(val))

    return math.floor(a)

,max_temp-1,max_temp-2,max_temp-3,max_temp-4,max_temp-5,max_temp-6,max_temp-7,max_temp-8,max_temp-9,max_temp-10,...,dailly_cases-1,dailly_cases-2,dailly_cases-3,dailly_cases-4,dailly_cases-5,dailly_cases-6,dailly_cases-7,dailly_cases-8,dailly_cases-9,dailly_cases-10
0,84,95,90,96,97,95,91,81,91,94,...,1043,1151,923,861,1020,771,944,1032,1155,846
1,89,84,95,90,96,97,95,91,81,91,...,907,1043,1151,923,861,1020,771,944,1032,1155
2,90,89,84,95,90,96,97,95,91,81,...,674,907,1043,1151,923,861,1020,771,944,1032
3,92,90,89,84,95,90,96,97,95,91,...,971,674,907,1043,1151,923,861,1020,771,944
4,82,92,90,89,84,95,90,96,97,95,...,957,971,674,907,1043,1151,923,861,1020,771
5,91,82,92,90,89,84,95,90,96,97,...,831,957,971,674,907,1043,1151,923,861,1020
6,82,91,82,92,90,89,84,95,90,96,...,1124,831,957,971,674,907,1043,1151,923,861
7,79,82,91,82,92,90,89,84,95,90,...,903,1124,831,957,971,674,907,1043,1151,923
8,85,79,82,91,82,92,90,89,84,95,...,880,903,1124,831,957,971,674,907,1043,1151
9,91,85,79,82,91,82,92,90,89,84,...,676,880,903,1124,831,957,971,674,907,1043


In [108]:
import submission
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error

## Parameters settings
past_cases_interval = 10
past_weather_interval = 10


## Read training data
train_file = './data/COVID_train_data.csv'
train_df = pd.read_csv(train_file)

## Read Training labels
train_label_file = './data/COVID_train_labels.csv'
train_labels_df = pd.read_csv(train_label_file)


## Read testing Features
test_fea_file = './data/test_features.csv'
test_features = pd.read_csv(test_fea_file)


## Set hyper-parameters for the SVM Model
svm_model = SVR()
svm_model.set_params(**{'kernel': 'rbf', 'degree': 1, 'C': 5000,
                        'gamma': 'scale', 'coef0': 0.0, 'tol': 0.001, 'epsilon': 10})


## Generate Prediction Results
predicted_cases_part1 = []
for idx in range(len(test_features)):
    test_feature = test_features.loc[idx]
    prediction = predict_COVID_part1(svm_model, train_df, train_labels_df, 
                                                past_cases_interval, past_weather_interval, test_feature)
    print(prediction)
    predicted_cases_part1.append(prediction)


print(predicted_cases_part1)

945
897


KeyboardInterrupt: 